# Plasticity with the Modified Cam-Clay model

In this section, we will introduce the implementation of plasticity which requires solving an *implicit* set of equations.

## Some recalls on plasticity

$$
f(\tens{\sigma}, \alpha) \leq 0
$$

## The Modified Cam-Clay (MCC) model

